# COVID-19: Descarga, procesamiento y visualización de datos oficiales en tiempo real

En esta práctica vamos a realizar un análisis de datos sobre la evolución de la pandemia y obtener representaciones gráficas de distintos aspectos.

Los datos que vamos a utilizar los descargamos directamente del [repositorio en GitHub del Center for Systems Science and Engineering (CSSE) del Hospital John Hopkins](https://github.com/CSSEGISandData/COVID-19) donde, a diario, se actualizan los datos obtenidos de diversas fuentes. De este modo, al cargar los datos los vamos a tener actualizados. 

Recomendamos acceder al repositorio mediante el enlace y comprobar cuándo  se han actualizado los datos (carpeta [csse_covid_19_data](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data) del repositorio). 

In [ ]:
# Obtenemos los datos
confirmados <- read.csv("https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")

recuperados <- read.csv("https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv")

muertos <- read.csv("https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv")

En los datos que acabamos de cargar tenemos

- una columna con la subregión (Provincia, Estado, etc)
- una columna con el país
- 2 columnas con latitud y longitud del lugar representado
- una cantidad (que se incrementa cada día) de columnas que representan fechas y contienen datos.

El formato de fecha es el anglosajón: mes/día/año.

In [ ]:
# Dimensión y estructura de los datos en confirmados
dim(confirmados)
str(confirmados)

## Preprocesado
Es importante cambiar los datos de formato serie de tiempo (con cada fila conteniendo observaciones de todos los días) a formato “tidy” (en cada fila una observación). 

Además, vamos a combinar los tres datasets en un único dataframe. Esto facilitará el análisis posterior al permitir filtrar los datos con facilidad por fecha y categoría.

Vamos a usar los paquetes `tidyverse` por sus funciones de transformación de datos y `lubridate` para manejar fechas. El paquete [tidyverse](https://www.tidyverse.org) contiene varias librerías para el tratamiento y análisis de datos.

In [ ]:
# Cargamos librerías
library(tidyverse)
library(lubridate)

Con el código que viene a continuación vamos a:

- Añadir a cada dataset una columna que indique lo que representa (confirmados, recuperados o muertes) y unirlos.
- Reordenar las columnas resultantes para tener primero las variables de lugar, coordenadas y categoría y, al final, las que representan fechas
- Reorganizar el dataframe para que las fechas ya no sean nombres de columnas sino valores de una nueva columna “Date”. 
- Agregar una columna “Value” que contenga el dats correspondiente a cada fecha
- Limpiar el valor de las fechas, eliminando la “X” que contienen al inicio

Almacenamos la información con el nombre `covid19`. En el código (mantenemos los nombres en inglés), gracias al paquete `tidyverse`, vamos añadiendo instrucciones con el operador *pipe* `%>%` que permite encadenar llamadas a funciones para así realizar de forma sencilla transformaciones de datos complejas.

Con esta nueva sintaxis (usando el operador pipe) ya no necesitamos anidar funciones, sino que las instrucciones van una después de otra. Es mucho más fácil de leer y de escribir.

In [ ]:
covid19 <- mutate(confirmados, Category = "Confirmed") %>% 
    bind_rows(mutate(recuperados, Category = "Recovered")) %>% 
    bind_rows(mutate(muertos, Category = "Deaths")) %>% 
    select(Province.State:Long, Category, everything()) %>% 
    pivot_longer(-(Province.State:Category), names_to = "Date", values_to = "Value") %>% 
    mutate(Date = mdy(str_remove(Date, "X")))

In [ ]:
# Vemos las primeras y últimas filas
head(covid19)
tail(covid19)

Tenemos entonces los datos en 7 columnas. En cada fila tenemos el dato `Value`  por fecha `Date` y categoría `Category` (confirmados, recuperados, muertos) para cada país `Country.Region`. Tenemos los datos geográficos (latitud `Lat` y longitud `Long` del país y, si procede, de la región/provincia/estado `Province.State` (Comunidad Autónoma en el caso de España).

Vemos ahora los valores NA presentes en nuestros datos.

In [ ]:
# Contamos valores NA en covid19
sum(is.na(covid19))

Puede ocurrir (y ocurre) que haya datos no actualizados o no disponibles por lo que nos aparecen valores nulos. Procedemos a eliminar los valores nulos ya que, en este caso, no parece conveniente realizar imputaciones (media, mediana, etc.).

In [ ]:
# Eliminamos valores nulos NA
covid19 <- covid19 %>% filter(!is.na(Value))

De este modo, podemos obtener los datos de una fecha concreta.

In [ ]:
# Datos a 10 de abril de 2021
covid19 %>% filter(Date == ymd("2020 04 10")) %>% head

In [ ]:
# Datos a última fecha disponible
covid19 %>%  filter(Date == max(Date)) %>% head

De este modo, con el código en R aplicado anteriormente, puede tener en su ordenador los **datos en tiempo real** del CSSE del Hospital John Hopkins.

### Evolución en el tiempo

Procedemos a visualizar la evolución de casos confirmados en España.

In [ ]:
# Evolución contagios España.
covid19  %>% 
  filter(Country.Region == 'Spain') %>% filter(Category == "Confirmed") %>%
  
  #Gráfico
  ggplot(aes(x= Date, y = Value, group = 1)) +
 geom_line(color="black")

Vemos una curva ascendente (excepto en un tramo correspondiente a febrero-marzo de 2021 del que hablaremos posteriormente) lo que indica que el número de casos confirmados es acumulado y no se proporcionan los casos diarios. Lo mismo ocurre con el número de recuperados y de muertos.

Podemos representar gráficamente esos 3 factores y compararlos por países. El siguiente código representa los gráficos correspondientes a España, Israel, Alemania y China.

In [ ]:
paises_a_comparar <- c("Spain", "Israel", "Germany", "India")

covid19 %>% 
    filter(Country.Region %in% paises_a_comparar) %>% 
    group_by(Category, Date, Country.Region) %>% 
    ggplot() +
    geom_col(aes(x = Date, y = Value, fill = Category)) +
    facet_wrap(~Country.Region, scales = "free") +
    scale_fill_brewer(palette = "Set2") +
    labs(title = "COVID19: cantidad de casos", 
         fill = "Situación",
         caption = "Fuente: 2019 Novel Coronavirus COVID-19 Data Repository by Johns Hopkins CSSE",
         y = NULL, x = NULL) +
    theme_minimal()

Vemos en una sola gráfica (una por país) los datos correspondientes a los 3 factores (confirmados, recuperados y muertos) de la variable `Category`.

**Ejercicio**: pruebe a cambiar los nombres de los países (en inglés) en el siguiente código y comparar los gráficos. Puede incluso incluir algún país más de los 4 presentes en el vector `paises_a_comparar` (no olvide la coma y las comillas).

In [ ]:
# Gráficos de otros países
paises_a_comparar2 <- c("Spain", "Israel", "Germany", "India")

covid19 %>% 
    filter(Country.Region %in% paises_a_comparar2) %>% 
    group_by(Category, Date, Country.Region) %>% 

    ggplot() +
    geom_col(aes(x = Date, y = Value, fill = Category)) +
    facet_wrap(~Country.Region, scales = "free") +
    scale_fill_brewer(palette = "Set2") +
    labs(title = "COVID19: cantidad de casos", 
         fill = "Situación",
         caption = "Fuente: 2019 Novel Coronavirus COVID-19 Data Repository by Johns Hopkins CSSE",
         y = NULL, x = NULL) +
    theme_minimal()

#### Comparación de casos confirmados en la misma escala
En las gráficas anteriores vemos que la escala de eje vertical difiere por países. Podemos representar los datos en un mismo gráfico.

In [ ]:
covid19 %>% 
    filter(Country.Region %in% paises_a_comparar, Category == "Confirmed") %>% 
    group_by(Date, Country.Region) %>% 
 summarise(Value = sum(Value)) %>% 
    ggplot() +
    geom_path(aes(x = Date, y = Value, color = Country.Region)) +
    labs(title = "COVID19: Infecciones confirmadas", 
         color = "",
         x = NULL, y = NULL,
         caption = "fuente: 2019 Novel Coronavirus COVID-19 Data Repository by Johns Hopkins CSSE") +
    theme_minimal()

Otra forma de unificar los valores de eje es utilizar una escala logarítmica.

In [ ]:
covid19 %>% 
    filter(Country.Region %in% paises_a_comparar, Category == "Confirmed") %>% 
    group_by(Date, Country.Region) %>% 
 summarise(Value = sum(Value)) %>% 
    ggplot() +
    geom_path(aes(x = Date, y = Value, color = Country.Region)) +
    scale_y_log10() +
    labs(title = "COVID19: Infecciones confirmadas", 
         color = "",
         x = NULL, y = NULL,
         caption = "fuente: 2019 Novel Coronavirus COVID-19 Data Repository by Johns Hopkins CSSE") +
    theme_minimal()

## ¿Errores en los datos de España?
Hemos visto en un gráfico anterior que el número de casos acumulados en España disminuía sobre una fecha de febrero/marzo de 2021. En las siguientes imágenes obtenidas a mediados de abril de 2021 de la web con los datos del John Hopkins vemos el mismo decrecimiento (al a derecha marcado con una flecha roja) en los casos acumulados en España sobre la fecha febrero/marzo de 2021.

<img src="_img/spain_data.png">

Fuente: [COVID-19 Dashboard](https://www.arcgis.com/apps/opsdashboard/index.html#/bda7594740fd40299423467b48e9ecf6)

Vamos a volver cargar desde el repositorio los datos acumulados de casos confirmados, almacenarlos con el nombre `spain`, filtar los datos de España, añadir la variable `incident_cases` con los casos diarios (restando a dato acumulado el anterior) y ver los datos correspondientes a los días 1, 2 y 3 de marzo de 2021.

In [ ]:
# Seleccionamos los datos de casos confirmados de España
#URL de los datos (github de la JHU)
jhu_url <- paste("https://raw.githubusercontent.com/CSSEGISandData/", 
                 "COVID-19/master/csse_covid_19_data/", "csse_covid_19_time_series/", 
                 "time_series_covid19_confirmed_global.csv", sep = "")

#Leemos los datos directamente de la fuente para tenerlos al dia

# Cambiamos nombres de columnas
spain <- read_csv(jhu_url) %>% 
  dplyr::rename("province" = "Province/State",
         "country_region" = "Country/Region") %>%
 
#Transformamos datos de "time series" a año-mes-día
  pivot_longer(-c(province,
                  country_region, 
                  Lat, 
                  Long), 
               names_to = "Date", 
               values_to = "cumulative_cases") %>% 
  
# Ajustamos el formato de fechas
  mutate(Date = mdy(Date)) %>% 
  
# Filtramos los datos de España (funciona para cualquier otro país).
  filter(country_region =="Spain") %>% 
  arrange(country_region, Date) %>% 
  group_by(country_region) %>% 

#Calculamos los casos diarios a partir de los datos acumulados
 
mutate(incident_cases = c(0, 
                            diff(cumulative_cases))) %>% 
  ungroup() %>% 
  
# Quitamos los datos de provincia, latitud y longitud.
# Estos son utiles si quieres hacer mapas,

  select(-c(province, Lat, Long))

# Y así quedan
tail(spain)

In [ ]:
# Representamos gráficamente los casos diarios (incident_cases)
spain %>%
  ggplot(aes(x= Date, y = incident_cases, group = 1)) +
 geom_line(color="black")

In [ ]:
# Datos de los días 1, 2 y 3 de marzo de 2021.  
spain %>% filter(Date == '2021-03-01')
spain %>% filter(Date == '2021-03-02')
spain %>% filter(Date == '2021-03-03')

Efectivamente, comprobamos que el día 2 de marzo de 2021 se notificaron 74347 casos menos. Sin entrar en otras disquisicioes, insistimos en la importancia de trabajar con datos fiables para obtener buenas conclusiones.

**Ejercicio**: utilice el siguiente bloque de código para obtener una gráfica con el número de fallecidos por covid en España. Almacene los datos en `spain2`. Puede copiar el código anterior y cambiar lo que corresponda para obtener y trabajar con los datos de fallecidos. Observe el resultado y obtenga sus propias conclusiones.

<details>    
<summary>
    <font size="3" color="black"><b>Solución</b></font>
</summary>
<p>
<ul>

jhu_url2 <- paste("https://raw.githubusercontent.com/CSSEGISandData/", 
                 "COVID-19/master/csse_covid_19_data/", "csse_covid_19_time_series/", 
                 "time_series_covid19_deaths_global.csv", sep = "")

spain2 <- read_csv(jhu_url2) %>% 
  dplyr::rename("province" = "Province/State",
         "country_region" = "Country/Region") %>%

pivot_longer(-c(province,
                  country_region, 
                  Lat, 
                  Long), 
               names_to = "Date", 
               values_to = "cumulative_deaths") %>% 
  

mutate(Date = mdy(Date)) %>% 
  

filter(country_region =="Spain") %>% 
arrange(country_region, Date) %>% 
group_by(country_region) %>% 

 
mutate(incident_deaths = c(0, 
                            diff(cumulative_deaths))) %>% 
  ungroup() %>% 
  

  select(-c(province, Lat, Long))
    
spain2 %>%
  ggplot(aes(x= Date, y = incident_deaths, group = 1)) +
 geom_line(color="black")
</ul>
</p>

## Incidencia acumulada. Medias móviles
Un dato que se está utilizando con frecuencia es el número de casos acumulados los últimos 7 o 14 días por 100000 habitantes.

Para calcular las medias móviles utilizamos la librería `RcppRoll`. La sintaxis para dicho cálculo em muy sencilla con la función `roll_mean`. Añadimos el el datafram `spain` como columnas a la derecha las medias móviles obtenidas.

In [ ]:
# Calculamos las medias móviles a 7 y 14 días
require(RcppRoll)

spain$mm7 <- roll_mean(spain$incident_cases, n = 7, align = "right", fill = NA)
spain$mm14 <- roll_mean(spain$incident_cases, n = 14, align = "right", fill = NA)
tail(spain)

In [ ]:
# Gráfica de la media móvil a 14 días
# Evidentemente, aparecen 13 errores
spain %>%
  ggplot(aes(x= Date, y = mm14, group = 1)) +
 geom_line(color="black") +
labs(title = "Medias móviles a 14 días (España)", 
         caption = "Fuente: 2019 Novel Coronavirus COVID-19 Data Repository by Johns Hopkins CSSE",
         y = NULL, x = NULL) +
    theme_minimal()